In [ ]:
!unzip /content/train.zip

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation

In [ ]:
data_dir = '/content/train'

In [46]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(28, 28),
    batch_size=32,
    class_mode='binary',
    subset='training',
    color_mode='rgb'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(28, 28),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    color_mode='rgb'
)

Found 164 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, MaxPooling2D, Flatten, Dense

inputs = Input(shape=(28, 28, 3))
x = MaxPooling2D(pool_size=(2, 2))(inputs)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dense(2, activation='softmax')(x)  # Используем 2 выхода для 2 классов

model = Model(inputs, x)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50
)

In [49]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation loss: {val_loss}, validation accuracy: {val_accuracy}')

2/2 [==============================] - 0s 11ms/step - loss: 0.6758 - accuracy: 0.6250
Validation loss: 0.6758458018302917, validation accuracy: 0.625


In [ ]:
model.save('model_rgb.h5')

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

test_image_path = '/content/test3.jpg'
img = load_img(test_image_path, target_size=(28, 28), color_mode='grayscale')  # Масштабирование и преобразование в ЧБ
img_array = img_to_array(img)  # Преобразование изображения в массив
img_array = np.expand_dims(img_array, axis=0)  # Добавление batch размерности
img_array /= 255.0  # Нормализация

predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions, axis=1)[0]  # Получение индекса наиболее вероятного класса

class_names = ['cat','dog']  # Убедитесь, что порядок соответствует порядку классов в вашей модели
predicted_class_name = class_names[predicted_class_index]

print(f"Predicted class: {predicted_class_name}")

1/1 [==============================] - 0s 13ms/step
Predicted class: dog


In [ ]:
!pip install -U coremltools


In [52]:
import coremltools as ct



image_input = ct.ImageType(shape=(1, 28, 28, 3))



coreml_model = ct.convert(
    model,
    inputs=[image_input],
    classifier_config=ct.ClassifierConfig(['cat', 'dog']),
    minimum_deployment_target=ct.target.iOS13
)


coreml_model.save('model_cat_dog_rgb.mlmodel')


Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 19/19 [00:00<00:00, 3871.92 ops/s]


In [ ]:
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
image_path = "/content/vitalik_resized_rgb.jpeg"
image = cv2.imread(image_path)

if image is None:
    print("Не удалось загрузить изображение")
else:
    cv2_imshow(image)

    height, width, channels = image.shape
    print("Ширина изображения:", width)
    print("Высота изображения:", height)
    print("Количество каналов изображения:", channels)
